EXAMPLE 1: Representing matrices as lists of lists, as done in a homework exercise.

Note that in "real life" you should use NumPy arrays instead of lists of lists.

In [ ]:
# Here we will represent a 3x3 matrix as a list of lists.
# The inner lists are the rows of the matrix.

# Note that Python source code lines are assumed to be continued when brackets are open.
# Hence the matrix A can be written in a visually obvious manner:
A=[[11,12,13],
   [21,22,23],
   [31,32,33]]

print("Matrix A as a list of lists:")
print(A)
print("The SECOND row of matrix A (remember that Python indices start at 0):")
print(A[1])
print("Element A_3,2:")
print(A[2][1])

# Report the trace of A, by summing the diagonal elements.
trA=0.0
for i in range(len(A)):
    trA+=A[i][i] # A[i][i] is the ith element of the ith row, where the numbering starts at 0.
print("Trace of A: {0:.15g}".format(trA))

# A function to add two matrices represented as lists of lists:
def add_matrices(A,B):
    """Return the sum of matrices A and B, where matrices are represented as lists of lists."""
    if len(A)!=len(B):
        raise ValueError("Matrices A and B should have the same number of rows.")
    C=[] # We will place the sum of matrices in C.
    for arow,brow in zip(A,B): # Loop over pairs of corresponding rows of A and rows of B.
        if len(arow)!=len(brow):
            raise ValueError("Matrices A and B should have the same numer of columns.")
        crow=[] # We will place each row of A+B in crow.
        for aelem,belem in zip(arow,brow): # Loop over pairs of corresponding elements in arow and brow.
            crow.append(aelem+belem)
        C.append(crow) # Add our new row to C.
    return C

print("Matrix A+A as a list of lists:")
print(add_matrices(A,A))
print()

import numpy as np
Anp=np.array(A) # Convert list of lists to a NumPy array.  Makes life a bit easier...

print("Matrix A recast as a NumPy array:")
print(Anp)
print("Second row of matrix A:")
print(Anp[1,:])
print("Element A_3,2:")
print(Anp[2,1])
print("Matrix 2A as a NumPy array:")
print(2*Anp)
print("Determinant of matrix A (using NumPy): {0:.15g}".format(np.linalg.det(Anp)))

EXAMPLE 2: Using NumPy.

In his book "Pillow Problems", Charles Dodgson (aka Lewis Carroll, who wrote the Alice stories) includes the following problem (no. 58):

       "Three Points are taken at random on an infinite Plane.  Find the chance of their being the vertices of an obtuse-angled Triangle."

In this example, we will solve this problem using a few different ways of choosing the random points in finite surfaces.

Initially, the code is set up to sample sets of three vertices uniformly in a square region.

(i) Read through the program and try to make sense of it.

(ii) Add a function three_points_in_rectangle that returns three random points uniformly distributed in a rectangle (whose aspect ratio you can choose).  Investigate whether the fraction of obtuse triangles depends on the aspect ratio of the rectangle.

(iii) Add a function three_points_in_circle that returns three random points uniformly distributed in the unit circle.  Hint: sample polar coordinates (r, theta), then convert the sampled coordinates to Cartesians.  Think carefully about the distribution of radial coordinates.  No doubt googling for "sample random points uniformly in circle" will suggest a solution.

(iv) Add functions for returning three random points on surfaces of your choice (e.g., surface of a sphere, triangular region, ...)

(v) Modify the program to produce a nice table comparing the fraction of obtuse triangles in different shapes of cell.  Are your results sufficiently precise that you can draw conclusions?  What are your conclusions regarding Charles Dodgson's original problem?

In [ ]:
import numpy as np

def obtuse(r1,r2,r3):
    """Return True if and only if vectors r1, r2 and r3 (NumPy arrays) are the vertices of an obtuse triangle."""
    # If a is the length of the side between vertices r1 and r2 then a^2=|r2-r1|^2=(r2-r1).(r2-r1).  Similarly for the other two sides, of length b and c.
    asq=(r2-r1).dot(r2-r1) ; bsq=(r3-r1).dot(r3-r1) ; csq=(r3-r2).dot(r3-r2)
    # Triangle is obtuse if the cosine of any of its angles is negative.  Use the cosine rule to test this.
    return asq+bsq<csq or bsq+csq<asq or csq+asq<bsq

def three_points_in_square():
    """Generate three random points in the 2D unit square 0 <= x < 1, 0 <= y < 1."""
    return np.random.random(2),np.random.random(2),np.random.random(2) # Generate three random 2D NumPy arrays.

def three_points_in_rectangle(aspect_ratio=1.0):
    """Generate three random points in a rectangle of unit area and aspect ratio aspect_ratio."""
    # Require ly/lx=aspect_ratio and lx.ly=1.
    lx=1.0/np.sqrt(aspect_ratio) ; ly=aspect_ratio*lx
    return tuple(np.array([np.random.random()*lx,np.random.random()*ly]) for _i in range(3))

def three_points_in_circle():
    """Generate three random points in the unit circle r<1."""
    pts=[]
    for _i in range(3):
        # Sample plane polar coordinates.
        theta=np.random.random()*2.0*np.pi # Polar angle is uniformly distributed on [0,2.pi).
        # Prob of r in range r to r+dr is 2.pi.r.dr / pi.1^2 = 2.r.dr.
        # Seek monotonic mapping from random variable u drawn from uniform distribution on interval [0,1) to
        # r in [0,1).  Matching infinitesimal probabilities, 2.r.dr = du.  Hence r^2 = u + const.  But u=0
        # corresponds to r=0.  So r^2 = u, i.e., r = sqrt(u).
        r=np.sqrt(np.random.random())
        # Return arrays containing positions in Cartesian coordinates.
        pts.append(np.array((r*np.cos(theta),r*np.sin(theta))))
    return tuple(pts)

def fraction_obtuse(nsamples=1000000,three_points=three_points_in_square):
    """Return the fraction of sampled triangles that are obtuse, where nsamples is the number of samples and
    three_points is the name of a function that chooses three random points."""
    nobtuse=0
    for _i in range(nsamples):
        r1,r2,r3=three_points()      # Sample the vertices of a triangle.
        nobtuse+=obtuse(r1,r2,r3)    # This uses a trick: True=1 and False=0.
    p=float(nobtuse)/float(nsamples) # Fraction of triangles that are obtuse.
    errp=np.sqrt(p*(1.0-p)/nsamples) # Approximate binomial distribution as normal distribution to estimate error in p.
    return p,errp

print("="*72)
print("Points scheme                   Fraction of obtuse triangles")
print("-"*72)
p,errp=fraction_obtuse(three_points=three_points_in_square)
print("Unit square                      {0:.8f} +/- {1:.8f}".format(p,errp))
for aspect_ratio in (1.0,2.0,5.0,10.0,20.0,50.0):
    # Use lambda keyword to create a function with no arguments that is equal to the function
    # three_points_in_rectangle with the given aspect_ratio.
    # (lambda is used to create functions without using "def", etc.)
    p,errp=fraction_obtuse(three_points=lambda: three_points_in_rectangle(aspect_ratio))
    print("Rectangle, asp ratio={0:4.2g}        {1:.8f} +/- {2:.8f}".format(aspect_ratio,p,errp))
p,errp=fraction_obtuse(three_points=three_points_in_circle)
print("Unit circle                      {0:.8f} +/- {1:.8f}".format(p,errp))
print("="*72)

**There is not a unique answer to Dodgson's problem.  Even if sets of three points are chosen uniformly, the fraction of obtuse triangles depends on the shape of the region within which the points are sampled.**